In [ ]:
import h5py
import numpy as np
#import cf_xarray as cfxr
import xarray 
import rioxarray
import holoviews as hv
from holoviews import opts
import geoviews as gv
import datashader as ds
import cartopy.crs as ccrs
import hvplot
from holoviews.operation.datashader import regrid, shade
from bokeh.tile_providers import STAMEN_TONER
import rasterio
from osgeo import gdal
from pathlib import Path
import panel
import hvplot.xarray
from ipywidgets import interact, Dropdown, FloatSlider, IntSlider, SelectMultiple, Text
import matplotlib.pyplot as plt
import xmltodict
import zipfile
import tarfile

hv.extension('bokeh', width=1000)

# EnMap 

## Extract files from EnMap package

In [ ]:
filename = '../../../data/EnMap/dims_op_oc_oc-en_700948309_1.tar.gz'
enmaptarfile = tarfile.open(filename, 'r')

In [ ]:
for i in enmaptarfile.getnames():
    if 'ZIP' in i:
        enmaptarfilezip = i

In [ ]:
enmaptarfile.extractfile(enmaptarfilezip)

In [ ]:
enmapzipfile = zipfile.ZipFile(enmaptarfilezip, 'r')

In [ ]:
enmapzipfile.filename

In [ ]:
enmaptarfile.extract(enmapzipfile.filename)

In [ ]:
for index, value in enumerate(enmapzipfile.filelist):
    if 'SPECTRAL_IMAGE' in str(value.filename):
        enmapspectralzipfile = value.filename
    if 'METADATA.XML' in str(value.filename):
        enmapmetadatazipfile = str(value.filename)

In [ ]:
filename = enmapzipfile.extract(enmapspectralzipfile)

In [ ]:
filenamexml = enmapzipfile.extract(enmapmetadatazipfile)

In [ ]:
fileptr = open(filenamexml,"r")

In [ ]:
xml_content= fileptr.read()

In [ ]:
enmapdict = xmltodict.parse(xml_content)

In [ ]:
vnirlist = enmapdict['level_X']['specific']['vnirProductQuality']['expectedChannelsList']

In [ ]:
swirlist = enmapdict['level_X']['specific']['swirProductQuality']['expectedChannelsList']

## Create useful band names and apply to GeoTIFF

In [ ]:
bands = []
for item in enmapdict['level_X']['specific']['bandCharacterisation']['bandID']:
    #print(item['bandNumber'], item['wavelengthCenterOfBand'])#, item['response'], item['wavelengths'], item['gainOfBand'], item['offsetOfBand'])
    bandnumber = (int(item['@number']))
    if str(bandnumber) in vnirlist:
        label = 'VNIR'
    else:
        label = 'SWIR'
    bands.append((bandnumber, label+str(bandnumber)+' '+str(int(item['wavelengthCenterOfBand'].split('.')[0]))))

In [ ]:
def set_band_descriptions(filepath, bands):
    """
    filepath: path/virtual path/uri to raster
    bands:    ((band, description), (band, description),...)
    """
    ds = gdal.Open(filepath, gdal.GA_Update)
    for band, desc in bands:
        rb = ds.GetRasterBand(band)
        rb.SetDescription(desc)
    del ds

In [ ]:
# Update the GeoTIFF band names to enable easier correspondence with other bands - use cwl for bands from metadata
set_band_descriptions(filename, bands)

In [ ]:
gdal.Info(filename).split('\n')

In [ ]:
geofilename = filename.replace('.TIF', '_geo.TIF')

In [ ]:
geofilename

## Warp GeoTIFF to lat/lon

In [ ]:
gdal.Warp(geofilename, filename, options="-t_srs EPSG:4326 -overwrite -tr 0.0003 0.0003 -tap")

In [ ]:
enmapxarray = xarray.open_dataset(filename)

In [ ]:
enmapxarraygeo = xarray.open_dataset(geofilename)

In [ ]:
enmapxarray

In [ ]:
(enmapxarray.band_data[[50,145,195]]/40).hvplot.rgb( geo=True, x='x', y='y', bands='band', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None, title="RGB Plot with HVPlot", width=1200)

In [ ]:
(enmapxarraygeo.band_data[[1,45,95]]/5).hvplot.rgb( geo=True, x='x', y='y', bands='band',  title="RGB Plot with HVPlot", width=1200, tiles='ESRI', alpha=0.5)

## Make an interactive RGB plot

In [ ]:
redW = Dropdown(options = list(enmapxarray.band.values))
greenW = Dropdown(options = list(enmapxarray.band.values))
blueW = Dropdown(options = list(enmapxarray.band.values))
startW = Dropdown(options = ['Pause', 'Go'])

@interact(red = redW, green = greenW, blue = blueW, start = startW)

def rgb_combo(red, green, blue, start):
    redW.options = list(enmapxarray.band.values)#prismaxarray.band.values.tolist()
    greenW.options = list(enmapxarray.band.values)#prismaxarray.band.values.tolist()
    blueW.options = list(enmapxarray.band.values)#prismaxarray.band.values.tolist()
    if start == 'Go' and red != green and green !=blue and red != blue:
        (enmapxarray[[red,green,blue]]/100).to_array().hvplot.rgb(x='x', y='y', bands='bands', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None)
        #enmapxarray.isel(band=[red,green,blue]).hvplot.rgb(x='x', y='y', bands='band', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None, title="RGB Plot with HVPlot", width=1200)
    return(red,green,blue,start)

In [ ]:
(enmapxarray.isel(band=[78,150,210])/50).hvplot.rgb(x='x', y='y', bands='band', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None, title="RGB Plot with HVPlot", width=1200)

In [ ]:
enmapxarray